In [ ]:
import sys
import os

#Use if working on Colab
#from google.colab import drive
#drive.mount('/content/drive')
#PATH = '/content/drive/My Drive/PPM_Stability/'

#If working locally
PATH = os.getcwd()
sys.path.append(PATH)

In [ ]:
#!pip install lime==0.2.0.1
#!pip install shap==0.35.0
#!pip install pandas==0.19.2
#!pip install xgboost==1.0.0

In [ ]:
import EncoderFactory
#from DatasetManager_for_colab import DatasetManager
from DatasetManager import DatasetManager
import BucketFactory

import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler

import time
import os
import sys
from sys import argv
import pickle
from collections import defaultdict
import random
import joblib

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from keras.models import Sequential, Model, load_model
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.layers import Dense, Embedding, Flatten, Input, LSTM
from keras.optimizers import Nadam, RMSprop
from keras.layers.normalization import BatchNormalization

from tensorflow.keras.backend import print_tensor
from tensorflow.keras.utils import plot_model
from tensorflow.compat.v1 import disable_v2_behavior#, ConfigProto, Session
from tensorflow.compat.v1.keras.backend import get_session
disable_v2_behavior()

import lime
import lime.lime_tabular

import shap

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

lime_pal = sns.diverging_palette(100, 200, s=150, as_cmap=True)
shap_pal = sns.diverging_palette(0, 240, s=150, as_cmap=True)

sns.set_style('whitegrid')

In [ ]:
dataset_ref = "production"
params_dir = PATH + "params"
results_dir = "results"
bucket_method = "single"
cls_encoding = "agg"
cls_method = "xgboost"

gap = 1
n_iter = 1

method_name = "%s_%s"%(bucket_method, cls_encoding)

generate_lime = True
generate_model_shap = True

sample_size = 0.25
exp_iter = 1
max_feat = 10

dataset_ref_to_datasets = {
    #"bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(5,6)],
    "bpic2017" : ["bpic2017_accepted"],
    "bpic2012" : ["bpic2012_accepted"],
    #"insurance": ["insurance_activity", "insurance_followup"],
    "sepsis_cases": ["sepsis_cases_1"],# "sepsis_cases_2", "sepsis_cases_4"]
    "production" : ["production"]
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]

datasets

In [ ]:
  for dataset_name in datasets:
      
      dataset_manager = DatasetManager(dataset_name)
      
      for ii in range(n_iter):
            num_buckets = len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))])

            all_pref_len = []
            all_feat_len = []
            all_shap_times = []
            all_lime_times = []
            sep_pref_len = []
            sep_feat_len = []
            all_times = []
            all_types = []

            sep_data_dict = {'Prefix Length': sep_pref_len, 'Feature Vector Length': sep_feat_len, 'SHAP Running Time': all_shap_times, 'LIME Running Time': all_lime_times}
            all_data_dict = {'Prefix Length': all_pref_len, 'Feature Vector Length': all_feat_len, 'Explainer': all_types, 'Running Time': all_times}
            sep_timing_path = os.path.join(PATH, "%s/%s_%s/sep_timing.csv" % (dataset_ref, cls_method, method_name))
            all_timing_path = os.path.join(PATH, "%s/%s_%s/all_timing.csv" % (dataset_ref, cls_method, method_name))


            for bucket in range(num_buckets):
                bucketID = bucket+1
                print ('Bucket', bucketID)

                pipeline_path = os.path.join(PATH, "%s/%s_%s/pipelines/pipeline_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                bucketer_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/bucketer_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))

                predictor = joblib.load(pipeline_path)
                cls = joblib.load(cls_path)
                feature_combiner = joblib.load(feat_comb_path)
                bucketer = joblib.load(bucketer_path)

                #import data for bucket
                X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))                  
                with open (X_train_path, 'rb') as f:
                    dt_train_bucket = pickle.load(f)

                X_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))                  
                with open (X_test_path, 'rb') as f:
                    dt_test_bucket = pickle.load(f)

                dt_test_bucket = dt_test_bucket.dropna()
                dt_test_sample = dt_test_bucket.sample(frac = sample_size)

                dt_testing_sample = feature_combiner.fit_transform(dt_test_sample)
                lens = dataset_manager.get_prefix_lengths(dt_test_sample)

                feat_len = [len(vec) for vec in dt_testing_sample]

                #Get a list of feature names
                feat_list = feature_combiner.get_feature_names()

                #create explainers now that can be passed later
                start_time = time.time()
                shap_explainer = shap.TreeExplainer(cls)
                duration = time.time() - start_time
                print("Time taken to create SHAP explainer:", duration, "seconds.")

                start_time = time.time()
                class_names=['regular','deviant']# regular is 0, deviant is 1, 0 is left, 1 is right
                trainingdata = feature_combiner.fit_transform(dt_train_bucket)
                lime_explainer = lime.lime_tabular.LimeTabularExplainer(trainingdata,
                                feature_names = feat_list,
                                class_names=class_names, discretize_continuous=True)
                duration = time.time() - start_time
                print("Time taken to create LIME explainer:", duration, "seconds.")


                shap_times = []
                lime_times = []

                for instance in dt_testing_sample:
                
                    #generate data for SHAP
                    start_time = time.time()
                    shap_explainer.shap_values(np.array([instance,]))
                    duration = time.time() - start_time
                    shap_times.append(duration)

                    #generate data for LIME
                    start_time = time.time()
                    lime_explainer.explain_instance(instance, cls.predict_proba)
                    duration = time.time() - start_time
                    lime_times.append(duration)

                #Update and save times
                sep_pref_len.extend(list(lens))
                sep_feat_len.extend(list(feat_len))
                all_shap_times.extend(shap_times)
                all_lime_times.extend(lime_times)

                for i in range(2):
                    all_pref_len.extend(sep_pref_len)
                    all_feat_len.extend(sep_feat_len)
                    
                all_times.extend(all_shap_times)
                all_times.extend(all_lime_times)
                all_types.extend(["SHAP"]*len(all_shap_times))
                all_types.extend(["LIME"]*len(all_lime_times))

            sep_data = pd.DataFrame(data = sep_data_dict)
            sep_data.to_csv(sep_timing_path, index = False)

            all_data = pd.DataFrame(data = all_data_dict)
            all_data.to_csv(all_timing_path, index = False)

In [ ]:
fig, ax = plt.subplots()
ax.plot(sep_data["Prefix Length"], sep_data["SHAP Running Time"], 'ro')#, y = ['SHAP Running Time', 'LIME Running Time'], x = ['Prefix Length'] )#, hue = data['Feature Vector Length'])
ax.plot(sep_data["Prefix Length"], sep_data["LIME Running Time"], 'bo')#, y = ['SHAP Running Time', 'LIME Running Time'], x = ['Prefix Length'] )#, hue = data['Feature Vector Length'])
#grid.map(sns.scatterplot, color=".3")
plt.show()


In [ ]:
sns.scatterplot(x = all_data['Prefix Length'], y = all_data['Running Time'], hue = all_data['Explainer'], size = all_data['Feature Vector Length'])
plt.show()